In [25]:
#Final KNN model to predict ETA
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset generated from previous steps
data = pd.read_excel(r"C:\Users\Admin\Downloads\New_data_cleaned.xlsx")

# Feature selection
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder_weather_a = LabelEncoder()
label_encoder_weather_b = LabelEncoder()

# Fit and transform the categorical data
data['Weather_A'] = label_encoder_weather_a.fit_transform(data['Weather_A'])
data['Weather_B'] = label_encoder_weather_b.fit_transform(data['Weather_B'])

# Select features and target variable
features = [
    'PointA_Lat', 'PointA_Lon', 'PointB_Lat', 'PointB_Lon',
    'Traffic_Density_A', 'Traffic_Density_B', 'Avg_Speed_A', 'Avg_Speed_B',
    'Distance_Haversine', 'Route_Distance',
] + [col for col in data.columns if col.startswith("Weather_")] + [col for col in data.columns if col.startswith("Temperature_")]

target = 'Travel_Time'

# Ensure no missing values
data = data.dropna()



# Split the data into training and testing sets
X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the KNN model
k = 5  # You can tune this value
knn = KNeighborsRegressor(n_neighbors=4, p=1, weights='distance')

# Fit the model
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")


Mean Squared Error: 36.54444062043974
R2 Score: 0.8273625509431874


In [41]:
import joblib

# Save the model
joblib.dump(knn, "knn_model.pkl")

['knn_model.pkl']

In [2]:
# Example: Make a prediction for a new data point
example_input = np.array([
    -33.85732632247613, 151.21382732395998,  # Example lat/lon for Point A
    -33.83065847575242, 151.15987822179764,  # Example lat/lon for Point B
    1, 1,          # Traffic density (normalized or scaled)
    70, 64,        # Average speeds
    5.798059729931215,  14.2934,        # Distance Haversine, Route Distance
    3,3,21.2,21.2    # Weather A , Weather B , Temparature A and Temprature B 
]).reshape(1, -1)

predicted_travel_time = knn.predict(example_input)
print(f"Predicted Travel Time: {predicted_travel_time[0]} minutes")

Predicted Travel Time: 17.653666250682765 minutes


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


In [45]:
import requests
import math
import joblib
import heapq
import numpy as np

# Function to fetch weather data
def fetch_weather(lat, lon):
    WEATHER_API_KEY = "178a1da68e9b4a0f8ba114734250501"
    url = f"https://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={lat},{lon}&aqi=yes"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp = data["current"]["temp_c"]
        condition = data["current"]["condition"]["text"]
        return temp if isinstance(temp, (int, float)) else 0, condition
    return 0, None

# Function to fetch traffic data
def fetch_traffic(lat, lon):
    TOMTOM_API_KEY = "EzGnjeDmKRjU4YjnJhEFUOUTziR1FqOf"
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?Key={TOMTOM_API_KEY}&Point={lat},{lon}&Unit=KMPH"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "flowSegmentData" in data:
            return float(data["flowSegmentData"].get("currentSpeed", 0)), float(data["flowSegmentData"].get("confidence", 0))
    return 0, 0

# Haversine function for calculating distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Function to fetch route details (using OSRM)
def fetch_route_details(lat1, lon1, lat2, lon2):
    OSRM_BASE_URL = "http://router.project-osrm.org/route/v1/driving/"
    coords = f"{lon1},{lat1};{lon2},{lat2}"
    url = OSRM_BASE_URL + coords
    params = {"overview": "false"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("routes"):
            route = data["routes"][0]
            distance = route["distance"] / 1000  # Convert meters to kilometers
            duration = route["duration"] / 60  # Convert seconds to minutes
            return distance, duration
    return 0, 0

# Function to load KNN model
def load_knn_model():
    return joblib.load("knn_model.pkl")

# Main function to create the graph with detailed node data
def create_turn_graph(start_coords, end_coords, knn_model):
    graph = {}

    for start, end in zip(start_coords, end_coords):
        lat1, lon1 = start
        lat2, lon2 = end

        # Fetch weather and traffic data for the start and end locations
        temp_a, weather_a = fetch_weather(lat1, lon1)
        temp_b, weather_b = fetch_weather(lat2, lon2)
        speed_a, density_a = fetch_traffic(lat1, lon1)
        speed_b, density_b = fetch_traffic(lat2, lon2)

        # Calculate Haversine distance for the route segment
        haversine_distance = haversine(lat1, lon1, lat2, lon2)

        # Fetch route distance and travel time
        route_distance, travel_time = fetch_route_details(lat1, lon1, lat2, lon2)

        # Define current node as the start of the segment
        current_node = (lat1, lon1)

        # Prepare node features (all the features required by the KNN model)
        node_features = {
            "Traffic_Density_A": density_a,
            "Traffic_Density_B": density_b,
            "Avg_Speed_A": speed_a,
            "Avg_Speed_B": speed_b,
            "Temperature_A": temp_a,
            "Weather_A": weather_a if weather_a else "Clear",  # Handle non-numeric weather
            "Temperature_B": temp_b,
            "Weather_B": weather_b if weather_b else "Clear",  # Handle non-numeric weather
            "Distance_Haversine": haversine_distance,
            "Route_Distance": route_distance,
            "PointA_Lat": lat1,
            "PointA_Lon": lon1,
            "PointB_Lat": lat2,
            "PointB_Lon": lon2
        }

        # Use KNN model to predict ETA for this edge (start -> end)
        try:
            eta = knn_model.predict([[
                node_features["Traffic_Density_A"], node_features["Traffic_Density_B"],
                node_features["Avg_Speed_A"], node_features["Avg_Speed_B"],
                node_features["Temperature_A"], node_features["Temperature_B"],
                node_features["Distance_Haversine"], node_features["Route_Distance"],
                node_features["Weather_A"], node_features["Weather_B"],
                node_features["PointA_Lat"], node_features["PointA_Lon"],
                node_features["PointB_Lat"], node_features["PointB_Lon"]
            ]])[0]  # KNN should return a single predicted ETA value
        except ValueError as e:
            print(f"Error predicting ETA: {e}")
            eta = 0  # Default ETA in case of error

        # Add the node and edge with predicted ETA
        if current_node not in graph:
            graph[current_node] = {}

        graph[current_node][(lat2, lon2)] = {
            "Traffic_Density_A": density_a,
            "Traffic_Density_B": density_b,
            "Avg_Speed_A": speed_a,
            "Avg_Speed_B": speed_b,
            "Temperature_A": temp_a,
            "Weather_A": weather_a,
            "Temperature_B": temp_b,
            "Weather_B": weather_b,
            "Distance_Haversine": haversine_distance,
            "Route_Distance": route_distance,
            "Travel_Time": eta  # Use ETA from KNN prediction
        }

    return graph

# Load your trained KNN model
knn_model = load_knn_model()

# Example start and end coordinates
start_coords = [(-33.8573, 151.2138)]  # Example start point (Sydney Opera House)
end_coords = [(-33.8307, 151.1599)]    # Example end point (Parramatta River)

# Create the graph using the KNN model for ETA prediction
graph = create_turn_graph(start_coords, end_coords, knn_model)

# Simple A* Algorithm using ETA as the cost function
def a_star(graph, start_node, goal_node):
    open_set = []  # Priority queue for open set (min-heap)
    heapq.heappush(open_set, (0, start_node))  # (priority, node)

    g_cost = {start_node: 0}  # Initialize the g_cost (cost from start)
    f_cost = {start_node: haversine(start_node[0], start_node[1], goal_node[0], goal_node[1])}  # Heuristic: Haversine distance

    came_from = {}

    while open_set:
        _, current_node = heapq.heappop(open_set)

        if current_node == goal_node:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start_node)
            return path[::-1]  # Return reversed path (from start to goal)

        for neighbor, features in graph.get(current_node, {}).items():
            eta = features["Travel_Time"]  # Use the predicted ETA as the edge cost
            new_g_cost = g_cost[current_node] + eta  # Accumulate cost with ETA

            if neighbor not in g_cost or new_g_cost < g_cost[neighbor]:
                g_cost[neighbor] = new_g_cost
                h_cost = haversine(neighbor[0], neighbor[1], goal_node[0], goal_node[1])  # Heuristic
                f_cost[neighbor] = new_g_cost + h_cost

                heapq.heappush(open_set, (f_cost[neighbor], neighbor))
                came_from[neighbor] = current_node

    return None  # No path found


# Run the A* algorithm on the constructed graph
start_node = (-33.8573, 151.2138)
goal_node = (-33.8307, 151.1599)

# Run the A* algorithm
path = a_star(graph, start_node, goal_node)

# Print the path found by A*
if path:
    print("\nPath found by A*:")
    for node in path:
        print(f"  {node}")
else:
    print("No path found")


Error predicting ETA: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.

Path found by A*:
  (-33.8573, 151.2138)
  (-33.8307, 151.1599)


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


In [47]:
#Final ANN code
import requests
import math
import joblib
from sklearn.preprocessing import LabelEncoder
import heapq

# Initialize label encoder for categorical features
label_encoder = LabelEncoder()

# Function to fetch weather data
def fetch_weather(lat, lon):
    WEATHER_API_KEY = "178a1da68e9b4a0f8ba114734250501"
    url = f"https://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={lat},{lon}&aqi=yes"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp = data["current"]["temp_c"]
        condition = data["current"]["condition"]["text"]
        return temp, condition
    return None, None

# Function to fetch traffic data
def fetch_traffic(lat, lon):
    TOMTOM_API_KEY = "EzGnjeDmKRjU4YjnJhEFUOUTziR1FqOf"
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?Key={TOMTOM_API_KEY}&Point={lat},{lon}&Unit=KMPH"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "flowSegmentData" in data:
            return data["flowSegmentData"].get("currentSpeed", "N/A"), data["flowSegmentData"].get("confidence", "N/A")
    return None, None

# Haversine function for calculating distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Function to fetch route details (using OSRM)
def fetch_route_details(lat1, lon1, lat2, lon2):
    OSRM_BASE_URL = "http://router.project-osrm.org/route/v1/driving/"
    coords = f"{lon1},{lat1};{lon2},{lat2}"
    url = OSRM_BASE_URL + coords
    params = {"overview": "false"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("routes"):
            route = data["routes"][0]
            distance = route["distance"] / 1000  # Convert meters to kilometers
            duration = route["duration"] / 60  # Convert seconds to minutes
            return distance, duration
    return None, None

# Function to load KNN model
def load_knn_model():
    return joblib.load("knn_model.pkl")

# Function to encode weather conditions as numerical values
def encode_weather_conditions(weather_conditions):
    return label_encoder.fit_transform(weather_conditions)

# Main function to create the graph with detailed node data
def create_turn_graph(start_coords, end_coords, knn_model):
    graph = {}

    for start, end in zip(start_coords, end_coords):
        lat1, lon1 = start
        lat2, lon2 = end

        # Fetch weather and traffic data for the start and end locations
        temp_a, weather_a = fetch_weather(lat1, lon1)
        temp_b, weather_b = fetch_weather(lat2, lon2)
        speed_a, density_a = fetch_traffic(lat1, lon1)
        speed_b, density_b = fetch_traffic(lat2, lon2)

        # Handle missing data with default or placeholder values
        temp_a = temp_a or 25
        temp_b = temp_b or 25
        weather_a = weather_a or "Clear"
        weather_b = weather_b or "Clear"
        speed_a = speed_a or 40
        density_a = density_a or 0.5
        speed_b = speed_b or 40
        density_b = density_b or 0.5

        # Encode weather conditions as numeric values
        weather_a_encoded = encode_weather_conditions([weather_a])[0]
        weather_b_encoded = encode_weather_conditions([weather_b])[0]

        # Calculate Haversine distance for the route segment
        haversine_distance = haversine(lat1, lon1, lat2, lon2)

        # Fetch route distance and travel time
        route_distance, travel_time = fetch_route_details(lat1, lon1, lat2, lon2)
        route_distance = route_distance or haversine_distance  # Fallback to Haversine distance

        # Define current node as the start of the segment
        current_node = (lat1, lon1)

        # Prepare node features (all the features required by the KNN model)
        node_features = [
            density_a, density_b,
            speed_a, speed_b,
            temp_a, temp_b,
            haversine_distance, route_distance,
            weather_a_encoded, weather_b_encoded,
            lat1, lon1, lat2, lon2
        ]

        # Use KNN model to predict ETA for this edge (start -> end)
        try:
            eta = knn_model.predict([node_features])[0]
        except ValueError as e:
            print(f"Error predicting ETA: {e}")
            eta = 9999  # Assign a very high ETA for invalid edges

        # Add the node and edge with predicted ETA
        if current_node not in graph:
            graph[current_node] = {}

        graph[current_node][(lat2, lon2)] = {
            "Traffic_Density_A": density_a,
            "Traffic_Density_B": density_b,
            "Avg_Speed_A": speed_a,
            "Avg_Speed_B": speed_b,
            "Temperature_A": temp_a,
            "Weather_A": weather_a_encoded,
            "Temperature_B": temp_b,
            "Weather_B": weather_b_encoded,
            "Distance_Haversine": haversine_distance,
            "Route_Distance": route_distance,
            "ETA": eta  # Use ETA from KNN prediction
        }

    return graph

# Load your trained KNN model
knn_model = load_knn_model()

# Example start and end coordinates
start_coords = [(-33.8573, 151.2138), (-33.8500, 151.2000)]  # Add more nodes
end_coords = [(-33.8500, 151.2000), (-33.8307, 151.1599)]

# Create the graph using the KNN model for ETA prediction
graph = create_turn_graph(start_coords, end_coords, knn_model)

# Simple A* Algorithm using ETA as the cost function
def a_star(graph, start_node, goal_node):
    open_set = []  # Priority queue for open set (min-heap)
    heapq.heappush(open_set, (0, start_node))  # (priority, node)

    g_cost = {start_node: 0}  # Initialize the g_cost (cost from start)
    f_cost = {start_node: haversine(start_node[0], start_node[1], goal_node[0], goal_node[1])}  # Heuristic: Haversine distance

    came_from = {}

    while open_set:
        _, current_node = heapq.heappop(open_set)

        if current_node == goal_node:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start_node)
            return path[::-1]  # Return reversed path (from start to goal)

        for neighbor, features in graph.get(current_node, {}).items():
            eta = features["ETA"]  # Use the predicted ETA as the edge cost
            new_g_cost = g_cost[current_node] + eta  # Accumulate cost with ETA

            if neighbor not in g_cost or new_g_cost < g_cost[neighbor]:
                g_cost[neighbor] = new_g_cost
                h_cost = haversine(neighbor[0], neighbor[1], goal_node[0], goal_node[1])  # Heuristic
                f_cost[neighbor] = new_g_cost + h_cost

                heapq.heappush(open_set, (f_cost[neighbor], neighbor))
                came_from[neighbor] = current_node

    return None  # No path found


# Run the A* algorithm on the constructed graph
start_node = (-33.8573, 151.2138)
goal_node = (-33.8307, 151.1599)

# Run the A* algorithm
path = a_star(graph, start_node, goal_node)

# Print the path found by A*
if path:
    print("\nPath found by A*:")
    for node in path:
        print(f"  {node}")
else:
    print("No path found")


SyntaxError: invalid syntax (279068508.py, line 1)